# Data Analysis: Methods and Stuff
As a starting point, I looked at two means of analysis: pearson r-correlations, and a self-organizing map. For each of these statistical analyses, I used the percent-only data set as a starting point.

## Pearson R-Correlations
A straightforward(ish) starting point, I used pearson r-correlations to see if any of the data points that we have for each metro area were correlated with each other. The easy part was running the correlation using open source tools. The hard part was deciding which correlations to run.

There are 250+ data points for each metro area. Running correlations on all of those means running 250^2 correlations, or 62,500 correlations.

Some of these could be simply ignored – correlating a data point with itself is unnecessary. Beyond those, narrowing the correlations to look at down further became a matter of subjective opinion on my part. Since we're mostly looking at these data points to look for evidence of if and how YIMBY and/or anti-displacement policies could work, I decided to start by doing only the following correlations:
* *data from different years* – i.e. a data point from 2014 couldn't be correlated with another data point from 2014. I made this choice to acknowledge that we're trying to see changes over time when looking at zoning/housing policies
* *data with different (enough) names* – some of the data points are called, for instance "2014 (GRAPHI) – 20.0 to 24.9 percent", and "2014 (GRAPHI) – 25.0 to 29.9 percent". These look similar, because this is the similar data point in in the same year for two rental brackets. We can assume that the distribution of rental brackets within the same year will correlate – while them not correlating would be interesting, it likely wouldn't yield any interesting insights on this conversation. So, we can at least start by not looking at similar-ish data. Eliminating correlations for similar data points was done by doing a comparison of data point names to see if they were similar using the `SequenceMatcher` below.
* *doing comparisons with rental data* – when looking at the YIMBY/anti-displacement conversation, one main thing to look for is trends in rental data. So, I started by looking specifically at correlations with rental data we had: median rent for 2-bedroom and 3-bedroom units, and the rental burden of tenants per year as a percentage of income. You can see this in the code below when it specifically looks in the column names for the phrases `"2BR", "3BR" and "GROSS RENT"`.

With these narrowed fields, I still have a couple thousand correlations to sift through, so that work is on-going.

```python
import csv
import difflib
from scipy.stats import pearsonr

all_data = {}
first = True

try:
    with open('/Users/Matt/Documents/sunshine-b/rent-const-occup-percent-only.csv', 'rU') as readfile:
        reader = csv.DictReader(readfile, dialect=csv.excel)
        for row in reader:
            for key in row.keys():
                if first:
                    all_data[key] = []
                try:
                    all_data[key].append(float(row[key]))
                except:
                    all_data[key].append('NA')
            if first:
                first = False
except OSError:
    print ("error getting input file")
finally:
    readfile.close()

keys = list(all_data.keys())
significant_r = {}

for i in range(len(keys)):
    for j in range(i, len(keys)):
        if '2014' in keys[i] and '2014' in keys[j]:
            continue
        elif '2015' in keys[i] and '2015' in keys[j]:
            continue
        elif '2016' in keys[i] and '2016' in keys[j]:
            continue
        if difflib.SequenceMatcher(None, keys[i], keys[j]).ratio() > 0.6:
            continue
        for word in ['3BR', '2BR', 'GROSS RENT']:
            if word in keys[i] or word in keys[j]:
                try:
                    r, p = pearsonr(all_data[keys[i]], all_data[keys[j]])
                    if p >= 0.00 and p <= 0.05 and r > 0.01:
                        key = '%s:%s' % (keys[i], keys[j])
                        significant_r[key] = {'r': r, 'p': p}
                except:
                    c1_exclude = [k for k in range(len(all_data[keys[i]])) if all_data[keys[i]][k] == 'NA']
                    c2_exclude = [k for k in range(len(all_data[keys[j]])) if all_data[keys[j]][k] == 'NA']
                    excludes = list(set(c1_exclude) | set(c2_exclude))
                    c1 = [k for k in range(len(all_data[keys[i]])) if k not in excludes]
                    c2 = [k for k in range(len(all_data[keys[j]])) if k not in excludes]
                    r, p = pearsonr(c1, c2)
                    if p > 0.00 and p <= 0.05:
                        key = '%s:%s' % (keys[i], keys[j])
                        significant_r[key] = {'r': r, 'p': p}
                break

with open('/Users/Matt/Documents/sunshine-b/percent-only-sig-values-v2.csv', 'w') as writefile:
    writer = csv.DictWriter(writefile, fieldnames=['key', 'r', 'p'])
    writer.writeheader()
    for key in significant_r.keys():
        writer.writerow({'key': key, 'r': significant_r[key]['r'], 'p': significant_r[key]['p']})
```

## Self-Organizing Map (SOM)
This is an AI technique for doing data clustering. Using a neural network, the algorithm goes through each metro area and its associated data points multiple times, each time doing a number of small associations between each data point and the data points of all other metro areas.

I am given the decision of figuring out ahead of time how many clusters I wish to have, and the clusters are set up as a grid. Deciding the number of clusters is pretty subjective – I decided to start by looking at a 4x4 grid, or 16 clusters.

To do the clustering, I put the percent-only data into a data mining tool I like to use called [Weka](https://www.cs.waikato.ac.nz/ml/weka/). This is a rapid prototyping tool that is set up to allow people automatically plug in `.csv` data (among other types of data) and immediately run various classification, clustering and association algorithms on the data to gain insight on it.

The results of the SOM run are in a `.csv` file in this notebook.